<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/vector-database-kata3-branch/kata3ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai


In [2]:
pip install openai faiss-cpu numpy


In [14]:
!pip uninstall -y faiss faiss-gpu
!pip install faiss-cpu


In [3]:
pip install openai faiss-cpu numpy

In [4]:
# Install necessary libraries (if not already installed)
!pip install --upgrade openai faiss-cpu numpy

In [5]:
import os
import openai
import faiss
import numpy as np
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('LLma')
openai.api_key = os.environ['OPENAI_API_KEY']

In [6]:
# Function to generate embeddings using OpenAI
def generate_embeddings(texts):
    try:
        # Ensure texts are in a list format
        if isinstance(texts, str):
            texts = [texts]

        # Generate embeddings
        response = openai.embeddings.create(
            model="text-embedding-ada-002",
            input=texts
        )

        # Extract embeddings properly
        embeddings = [item.embedding for item in response.data]

        # Convert to numpy array with float32 dtype (FAISS expects this)
        return np.array(embeddings).astype('float32')

    except Exception as e:
        print(f"Error generating embeddings: {e}")
        return None



In [7]:
# Example corpus with more documents
documents = [
    "The capital of France is Paris.",
    "Python is a programming language.",
    "The Eiffel Tower is in Paris.",
    "Machine learning is a subset of artificial intelligence.",
    "The Great Wall of China is a famous landmark.",
    "Photosynthesis is the process plants use to make food.",
    "Mount Everest is the tallest mountain in the world.",
    "Water boils at 100 degrees Celsius.",
    "Shakespeare wrote Romeo and Juliet.",
    "The moon orbits the Earth."
]

In [9]:

# Generate embeddings for the documents
document_embeddings = generate_embeddings(documents)

if document_embeddings is None:
    print("Failed to generate document embeddings. Exiting...")
    exit()



In [10]:
# Check type, dtype, and shape again
print("Type:", type(document_embeddings))
print("Dtype:", document_embeddings.dtype)
print("Shape:", document_embeddings.shape)

# Ensure it's properly converted to a contiguous numpy array of float32
document_embeddings = np.array(document_embeddings, dtype=np.float32)
document_embeddings = np.ascontiguousarray(document_embeddings)

# Check again after conversion
print("After conversion:")
print("Type:", type(document_embeddings))
print("Dtype:", document_embeddings.dtype)
print("Shape:", document_embeddings.shape)

# Try adding to FAISS index again
index = faiss.IndexFlatL2(document_embeddings.shape[1])
index.add(document_embeddings)

print("Embeddings added to FAISS index successfully!")


Type: <class 'numpy.ndarray'>
Dtype: float32
Shape: (10, 1536)
After conversion:
Type: <class 'numpy.ndarray'>
Dtype: float32
Shape: (10, 1536)
Embeddings added to FAISS index successfully!


In [11]:
import faiss
print(faiss.__version__)


1.10.0


In [13]:
# Continuous conversation chatbot
def chatbot():
    print("Chatbot: Hello! Ask me anything or type 'exit' to quit.")

    conversation_history = []

    while True:
        # User input
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break

        if not user_input:
            print("Chatbot: Please enter a question.")
            continue

        # Generate embedding for the query
        query_embedding = generate_embeddings(user_input)
        if query_embedding is None:
            print("Chatbot: Failed to generate query embedding.")
            continue

        # Perform the search on the FAISS index
        k = 3  # Number of nearest neighbors to retrieve
        D, I = index.search(query_embedding, k)

        # Respond with the most relevant document
        print("Chatbot: Here’s what I found:")
        for i, doc_index in enumerate(I[0]):
            doc = documents[doc_index]
            score = D[0][i]
            print(f"- {doc} (Score: {score:.4f})")
        print("-" * 50)


# Run the chatbot
chatbot()


Chatbot: Hello! Ask me anything or type 'exit' to quit.
You: what is the capital of france
Chatbot: Here’s what I found:
- The capital of France is Paris. (Score: 0.1527)
- The Eiffel Tower is in Paris. (Score: 0.3604)
- The Great Wall of China is a famous landmark. (Score: 0.5321)
--------------------------------------------------
You: tell about machine learning
Chatbot: Here’s what I found:
- Machine learning is a subset of artificial intelligence. (Score: 0.2745)
- Python is a programming language. (Score: 0.4452)
- The moon orbits the Earth. (Score: 0.5591)
--------------------------------------------------
You: which is the tallest mountain in the world
Chatbot: Here’s what I found:
- Mount Everest is the tallest mountain in the world. (Score: 0.1920)
- The Great Wall of China is a famous landmark. (Score: 0.4145)
- The Eiffel Tower is in Paris. (Score: 0.4713)
--------------------------------------------------
You: exit
Chatbot: Goodbye!
